In [11]:
import pandas as pd
import numpy as np

# Number of samples taken
sample_amount = 1000

# Files
file_path = '../data/20231002-20231106/20231002-20231106/20230501-20230605/20230501-20230605/20230501.csv'
battle_file = 'battle_stats.csv'
card_file = 'card_stats.csv'

# Load the data
data = pd.read_csv(file_path)

# Defining Column Names
new_column_names = ['Time', 'Gamemode', 'P1Tag', 'P1Trophies', 'P1Crowns', 'P1Card1', 'P1Card2', 'P1Card3', 'P1Card4', 'P1Card5', 'P1Card6', 'P1Card7', 'P1Card8', 'P2Tag', 'P2Trophies', 'P2Crowns', 'P2Card1', 'P2Card2', 'P2Card3', 'P2Card4', 'P2Card5', 'P2Card6', 'P2Card7', 'P2Card8']

# Assign new column names
data.columns = new_column_names

# Check if the file has at least sample_amount rows
if len(data) >= sample_amount:
    sample = data.sample(n=sample_amount, random_state=np.random.RandomState())
    
    # Write the sample to a new CSV file
    sample.to_csv(battle_file, index=False)
    
    print(f"Sampled data written to {battle_file}")
else:
    print(f"The file only has {len(data)} rows, less than {sample_amount}.")

Sampled data written to battle_stats.csv


In [12]:
# Add win column

data = sample

# Create PLayer 1 win column
data['P1Win'] = data.iloc[:, 4] > data.iloc[:, 15]

# Save the updated DataFrame to a new CSV file
data.to_csv(battle_file, index=False)

print(f"File has been updated and saved as {battle_file}.")


File has been updated and saved as battle_stats.csv.


In [13]:
import json

# List of all card columns for both players
card_columns_p1 = [f'P1Card{j}' for j in range(1, 9)]
card_columns_p2 = [f'P2Card{j}' for j in range(1, 9)]

# Initialize a dictionary to store card stats
card_stats = {}

# Function to update stats
def update_stats(card, player_won):
    if card not in card_stats:
        card_stats[card] = {'played': 0, 'wins': 0}
    card_stats[card]['played'] += 1
    if player_won:
        card_stats[card]['wins'] += 1

# Iterate over each row and update stats
for _, row in data.iterrows():
    p1_won = row['P1Win']
    for card in row[card_columns_p1]:
        update_stats(card, p1_won)
    for card in row[card_columns_p2]:
        update_stats(card, not p1_won)

# Calculate additional statistics
total_picks = sum([stat['played'] for stat in card_stats.values()])
for card, stats in card_stats.items():
    stats['pick_rate'] = stats['played'] / total_picks
    stats['win_rate'] = stats['wins'] / stats['played'] if stats['played'] > 0 else 0

# Load the JSON file with card ID-to-name mapping
json_file_path = 'card_info.json'  # Replace with your JSON file path
with open(json_file_path, 'r', encoding="utf-8") as file:
    card_name_mapping = json.load(file)

# Convert to DataFrame
stats_df = pd.DataFrame.from_dict(card_stats, orient='index')

# Add Cards Name to column
stats_df.index = stats_df.index.astype(str)
stats_df.index = stats_df.index.map(card_name_mapping)
stats_df.index.name = 'Cards'

# Save to a new CSV file
stats_df.to_csv(card_file)

print(f"Card statistics written to {card_file}")

Card statistics written to card_stats.csv
